In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import os
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion
from gammapy.spectrum import (
    SpectrumDatasetOnOff,
    SpectrumDataset,
    SpectrumDatasetMaker,
    FluxPointsEstimator,
    FluxPointsDataset,
    ReflectedRegionsBackgroundMaker,
    plot_spectrum_datasets_off_regions,
)
from gammapy.modeling import Fit, Parameter, Datasets
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    SpectralModel,
    SkyModel,
    ExpCutoffPowerLawSpectralModel,
)
from gammapy.irf import load_cta_irfs
from gammapy.data import Observation
from gammapy.maps import MapAxis
from itertools import combinations

In [3]:
import scipy.stats as stats
import math
import statistics

In [4]:
os.environ['CALDB'] = '/home/rishank/anaconda2/envs/cta/share/caldb/'
!echo $CALDB
!ls $CALDB

/home/rishank/anaconda2/envs/cta/share/caldb/
data


In [5]:
irfs = load_cta_irfs(
    "$CALDB/data/cta/prod3b-v2/bcf/South_z20_50h/irf_file.fits"
)

In [6]:
l = [0,0.02,0.05,0.1,0.5]
livetime = [10,8,5,4,2] * u.h
n_obs = [10,10,10,10,10]
model_name = ['PowerLaw', 'Cutoff_value \n= 50TeV', 'Cutoff_value \n= 20TeV', 'Cutoff_value \n= 10TeV', 'Cutoff_value \n= 2TeV']
pointing = SkyCoord(0, 0, unit="deg", frame="galactic")
offset = 0.5 * u.deg
# Reconstructed and true energy axis
energy_axis = MapAxis.from_edges(
    np.logspace(-1.5, 2.0, 40), unit="TeV", name="energy", interp="log"
)
energy_axis_true = MapAxis.from_edges(
    np.logspace(-1.5, 2.0, 200), unit="TeV", name="energy", interp="log"
)

on_region_radius = Angle("0.11 deg")
on_region = CircleSkyRegion(center=pointing, radius=on_region_radius)

In [7]:
rows, cols = (5, 5) 
model = [[0 for i in range(cols)] for j in range(rows)]
print(model)

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


In [8]:
simu = []
for j in range(5):
    for i in range(5):
        sim = ExpCutoffPowerLawSpectralModel(
            index=2.22,
            amplitude=1.289e-12 * u.Unit("cm-2 s-1 TeV-1"),
            reference=1 * u.TeV,
            lambda_=l[j] * u.Unit("TeV-1"),
            alpha = 1,
        )
        mod = SkyModel(spectral_model=sim)
        model[i][j]=mod
    simu.append(sim)
    print(simu[j])

ExpCutoffPowerLawSpectralModel

   name     value   error      unit      min max frozen
--------- --------- ----- -------------- --- --- ------
    index 2.220e+00   nan                nan nan  False
amplitude 1.289e-12   nan cm-2 s-1 TeV-1 nan nan  False
reference 1.000e+00   nan            TeV nan nan   True
  lambda_ 0.000e+00   nan          TeV-1 nan nan  False
    alpha 1.000e+00   nan                nan nan   True
ExpCutoffPowerLawSpectralModel

   name     value   error      unit      min max frozen
--------- --------- ----- -------------- --- --- ------
    index 2.220e+00   nan                nan nan  False
amplitude 1.289e-12   nan cm-2 s-1 TeV-1 nan nan  False
reference 1.000e+00   nan            TeV nan nan   True
  lambda_ 2.000e-02   nan          TeV-1 nan nan  False
    alpha 1.000e+00   nan                nan nan   True
ExpCutoffPowerLawSpectralModel

   name     value   error      unit      min max frozen
--------- --------- ----- -------------- --- --- ------
    inde

In [9]:
obs = [[0 for i in range(cols)] for j in range(rows)]
for j in range(5):
    for i in range(5):
        obs[i][j]=Observation.create(pointing=pointing, livetime=livetime[i], irfs=irfs)
        print(obs[i][j])

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 36000.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 28800.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 18000.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 14400.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 7200.0 s



Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 36000.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 28800.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 18000.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 14400.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 7200.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 36000.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 28800.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 18000.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 14400.0 s

Info for OBS_ID = 1
- Pointing pos: RA 266.40 deg / Dec -28.94 deg
- Livetime duration: 7200.0 s

Info for OB

In [10]:
dataset = [[0 for i in range(cols)] for j in range(rows)]
for j in range(5):
    for i in range(5):
        dataset_empty = SpectrumDataset.create(
            e_reco=energy_axis.edges, e_true=energy_axis_true.edges, region=on_region
        )
        maker = SpectrumDatasetMaker(selection=["aeff", "edisp", "background"])
        dataset[i][j]=maker.run(dataset_empty, obs[i][j])
        print(dataset[i][j])

SpectrumDataset

    Name                            : 1 

    Total counts                    : nan 
    Total predicted counts          : nan
    Total background counts         : 4417.82

    Effective area min              : 2.80e+04 m2
    Effective area max              : 5.73e+06 m2

    Livetime                        : 3.60e+04 s

    Number of total bins            : 0 
    Number of fit bins              : 39 

    Fit statistic type              : cash
    Fit statistic value (-2 log(L)) : nan

    Number of parameters            : 0
    Number of free parameters       : 0


SpectrumDataset

    Name                            : 1 

    Total counts                    : nan 
    Total predicted counts          : nan
    Total background counts         : 3534.26

    Effective area min              : 2.80e+04 m2
    Effective area max              : 5.73e+06 m2

    Livetime                        : 2.88e+04 s

    Number of total bins            : 0 
    Number of fit bins 

SpectrumDataset

    Name                            : 1 

    Total counts                    : nan 
    Total predicted counts          : nan
    Total background counts         : 883.56

    Effective area min              : 2.80e+04 m2
    Effective area max              : 5.73e+06 m2

    Livetime                        : 7.20e+03 s

    Number of total bins            : 0 
    Number of fit bins              : 39 

    Fit statistic type              : cash
    Fit statistic value (-2 log(L)) : nan

    Number of parameters            : 0
    Number of free parameters       : 0


SpectrumDataset

    Name                            : 1 

    Total counts                    : nan 
    Total predicted counts          : nan
    Total background counts         : 4417.82

    Effective area min              : 2.80e+04 m2
    Effective area max              : 5.73e+06 m2

    Livetime                        : 3.60e+04 s

    Number of total bins            : 0 
    Number of fit bins  

In [11]:
data = [[0 for i in range(cols)] for j in range(rows)]
for j in range(5):
    for i in range(5):
        dataset[i][j].model = model[i][j]
        dataset[i][j].fake(random_state=42)
        print(dataset[i][j])
        data[i][j]=dataset[i][j]

SpectrumDataset

    Name                            : 1 

    Total counts                    : 4378 
    Total predicted counts          : nan
    Total background counts         : 4417.82

    Effective area min              : 2.80e+04 m2
    Effective area max              : 5.73e+06 m2

    Livetime                        : 3.60e+04 s

    Number of total bins            : 39 
    Number of fit bins              : 39 

    Fit statistic type              : cash
    Fit statistic value (-2 log(L)) : nan

    Number of parameters            : 0
    Number of free parameters       : 0


SpectrumDataset

    Name                            : 1 

    Total counts                    : 3498 
    Total predicted counts          : nan
    Total background counts         : 3534.26

    Effective area min              : 2.80e+04 m2
    Effective area max              : 5.73e+06 m2

    Livetime                        : 2.88e+04 s

    Number of total bins            : 39 
    Number of fit b

In [12]:
data_onoff = [[0 for i in range(cols)] for j in range(rows)]
for j in range(5):
    for i in range(5):
        dataset_onoff = SpectrumDatasetOnOff(
            aeff=data[i][j].aeff,
            edisp=data[i][j].edisp,
            models=model[i][j],
            livetime=livetime[i],
            acceptance=1,
            acceptance_off=5,
        )
        dataset_onoff.fake(background_model=data[i][j].background)
        print(dataset_onoff)
        data_onoff[i][j]=dataset_onoff

SpectrumDatasetOnOff

    Name                            :  

    Total counts                    : 7983 
    Total predicted counts          : 7934.69
    Total off counts                : 22232.00

    Total background counts         : 4446.40

    Effective area min              : 2.80e+04 m2
    Effective area max              : 5.73e+06 m2

    Livetime                        : 1.00e+01 h

    Number of total bins            : 39 
    Number of fit bins              : 39 

    Fit statistic type              : wstat
    Fit statistic value (-2 log(L)) : 43.46

    Number of parameters            : 5
    Number of free parameters       : 3

    Model type                      : SkyModels
    Acceptance mean:                : 1.0

SpectrumDatasetOnOff

    Name                            :  

    Total counts                    : 6444 
    Total predicted counts          : 6367.23
    Total off counts                : 17883.00

    Total background counts         : 3576.60

    Eff

SpectrumDatasetOnOff

    Name                            :  

    Total counts                    : 7581 
    Total predicted counts          : 7610.80
    Total off counts                : 22119.00

    Total background counts         : 4423.80

    Effective area min              : 2.80e+04 m2
    Effective area max              : 5.73e+06 m2

    Livetime                        : 1.00e+01 h

    Number of total bins            : 39 
    Number of fit bins              : 39 

    Fit statistic type              : wstat
    Fit statistic value (-2 log(L)) : 29.25

    Number of parameters            : 5
    Number of free parameters       : 3

    Model type                      : SkyModels
    Acceptance mean:                : 1.0

SpectrumDatasetOnOff

    Name                            :  

    Total counts                    : 5947 
    Total predicted counts          : 6077.40
    Total off counts                : 17639.00

    Total background counts         : 3527.80

    Eff

In [13]:
%%time
datas = [[0 for i in range(cols)] for j in range(rows)]
for j in range(5):
    for i in range(5):
        datasets = []
        for idx in range(n_obs[i]):
            data_onoff[i][j].fake(random_state=idx, background_model=data[i][j].background)
            data_onoff[i][j].name = f"obs_{idx}"
            datasets.append(data_onoff[i][j].copy())
        datas[i][j]=datasets

CPU times: user 1.61 s, sys: 47.5 ms, total: 1.66 s
Wall time: 1.7 s


In [14]:
for j in range(5):
    for i in range(5):
        n_on = [dataset.counts.data.sum() for dataset in datas[i][j]]
        n_off = [dataset.counts_off.data.sum() for dataset in datas[i][j]]
        excess = [dataset.excess.data.sum() for dataset in datas[i][j]]
        
        fix, axes = plt.subplots(1, 3, figsize=(12, 4))
        axes[0].hist(n_on)
        axes[0].set_xlabel("no. of counts\nn_on")
        axes[0].set_ylabel("No. of observations")
        axes[1].hist(n_off)
        axes[1].set_xlabel("no. of counts\nn_off")
        axes[1].set_ylabel("No. of observations")
        axes[2].hist(excess)
        axes[2].set_xlabel("no. of counts\nexcess");
        axes[2].set_ylabel("No. of observations")

KeyboardInterrupt: 

<Figure size 864x288 with 0 Axes>

In [ ]:
res1 = [[0 for i in range(cols)] for j in range(rows)]
res2 = [[0 for i in range(cols)] for j in range(rows)]
e_edges = np.logspace(-1.5, 2.0, 10) * u.TeV
spectra = ExpCutoffPowerLawSpectralModel(index=2, amplitude=2e-11 * u.Unit("cm-2 s-1 TeV-1"), reference=1 * u.TeV, lambda_=0 * u.Unit("TeV-1"), alpha = 1)
#spectra = PowerLawSpectralModel(index=2, amplitude=2e-11 * u.Unit("cm-2 s-1 TeV-1"), reference=1 * u.TeV)
profile1 = [[0 for i in range(cols)] for j in range(rows)]
profile2 = [[0 for i in range(cols)] for j in range(rows)]
model_0 = SkyModel(spectral_model=spectra)
for j in range(5):
    for i in range(5):
        for dataset in datas[i][j]:
            dataset.models = model_0

In [ ]:
%%time
for j in range(5):
    for i in range(5):
        results = []
        profile = []
        for dataset in datas[i][j]:
            dataset.parameters["lambda_"].frozen = True
            fit = Fit([dataset])
            result = fit.run()
            results.append(
                {
                    "index": result.parameters["index"].value,
                    "amplitude": result.parameters["amplitude"].value,
                    "lambda_": result.parameters["lambda_"].value,
                    "reference":result.parameters["reference"].value,
                    "alpha":result.parameters["alpha"].value,
                    "TS":result.total_stat,
                }
            )
            profile.append(fit.stat_profile(parameter="index"))
            print(result.parameters.to_table())
        profile1[i][j] = profile
        res1[i][j]=results

In [ ]:
%%time
for j in range(5):
    for i in range(5):
        results = []
        profile = []
        for dataset in datas[i][j]:
            dataset.parameters["lambda_"].frozen = False
            fit = Fit([dataset])
            result = fit.run()
            results.append(
                {
                    "index": result.parameters["index"].value,
                    "amplitude": result.parameters["amplitude"].value,
                    "lambda_": result.parameters["lambda_"].value,
                    "reference":result.parameters["reference"].value,
                    "alpha":result.parameters["alpha"].value,
                    "TS":result.total_stat,
                }
            )
            profile.append(fit.stat_profile(parameter="index"))
            print(result.parameters.to_table())
        profile2[i][j] = profile
        res2[i][j]=results

In [ ]:
index1 = [[0 for i in range(cols)] for j in range(rows)]
amplitude1 = [[0 for i in range(cols)] for j in range(rows)]
lambda_1 = [[0 for i in range(cols)] for j in range(rows)]
index2 = [[0 for i in range(cols)] for j in range(rows)]
amplitude2 = [[0 for i in range(cols)] for j in range(rows)]
lambda_2 = [[0 for i in range(cols)] for j in range(rows)]
TS1 = [[0 for i in range(cols)] for j in range(rows)]
TS2 = [[0 for i in range(cols)] for j in range(rows)]

for j in range(5):
    for i in range(5):
        a = np.array([_["index"] for _ in res1[i][j]])
        b = np.array([_["amplitude"] for _ in res1[i][j]])
        c = np.array([_["lambda_"] for _ in res1[i][j]])
        d = np.array([_["index"] for _ in res2[i][j]])
        e = np.array([_["amplitude"] for _ in res2[i][j]])
        f = np.array([_["lambda_"] for _ in res2[i][j]])
        g = np.array([_["TS"] for _ in res1[i][j]])
        h = np.array([_["TS"] for _ in res2[i][j]])
        index1[i][j]=a
        amplitude1[i][j]=b
        lambda_1[i][j]=c
        index2[i][j]=d
        amplitude2[i][j]=e
        lambda_2[i][j]=f
        TS1[i][j]=g
        TS2[i][j]=h

In [ ]:
del_TS = [[0 for i in range(cols)] for j in range(rows)]
for j in range(5):
    for i in range(5):
        del_TS[i][j] = TS1[i][j] - TS2[i][j]

In [ ]:
fig = plt.figure(figsize=[15,20],constrained_layout=True)

import matplotlib.gridspec as gridspec

gs0 = gridspec.GridSpec(1, 5, figure=fig)

gs1 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[0])
for n in range(5):
    ax = fig.add_subplot(gs1[n])
    plt.hist(index1[n][0], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["index"].value, color="green")
    plt.xlabel('Index\nl=0')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs2 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[1])
for n in range(5):
    ax = fig.add_subplot(gs2[n])
    plt.hist(index1[n][1], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["index"].value, color="green")
    plt.xlabel('Index\nl=0.02')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs3 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[2])
for n in range(5):
    ax = fig.add_subplot(gs3[n])
    plt.hist(index1[n][2], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["index"].value, color="green")
    plt.xlabel('Index\nl=0.05')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs4 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[3])
for n in range(5):
    ax = fig.add_subplot(gs4[n])
    plt.hist(index1[n][3], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["index"].value, color="green")
    plt.xlabel('Index\nl=0.1')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs5 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[4])
for n in range(5):
    ax = fig.add_subplot(gs5[n])
    plt.hist(index1[n][4], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["index"].value, color="green")
    plt.xlabel('Index\nl=0.5')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
plt.show()

In [ ]:
fig = plt.figure(figsize=[15,20],constrained_layout=True)

import matplotlib.gridspec as gridspec

gs0 = gridspec.GridSpec(1, 5, figure=fig)

gs1 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[0])
for n in range(5):
    ax = fig.add_subplot(gs1[n])
    plt.hist(index2[n][0], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["index"].value, color="green")
    plt.xlabel('Index\nl=0')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs2 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[1])
for n in range(5):
    ax = fig.add_subplot(gs2[n])
    plt.hist(index2[n][1], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["index"].value, color="green")
    plt.xlabel('Index\nl=0.02')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs3 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[2])
for n in range(5):
    ax = fig.add_subplot(gs3[n])
    plt.hist(index2[n][2], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["index"].value, color="green")
    plt.xlabel('Index\nl=0.05')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs4 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[3])
for n in range(5):
    ax = fig.add_subplot(gs4[n])
    plt.hist(index2[n][3], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["index"].value, color="green")
    plt.xlabel('Index\nl=0.1')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs5 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[4])
for n in range(5):
    ax = fig.add_subplot(gs5[n])
    plt.hist(index2[n][4], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["index"].value, color="green")
    plt.xlabel('Index\nl=0.5')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
plt.show()

In [ ]:
fig = plt.figure(figsize=[15,20],constrained_layout=True)

import matplotlib.gridspec as gridspec

gs0 = gridspec.GridSpec(1, 5, figure=fig)

gs1 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[0])
for n in range(5):
    ax = fig.add_subplot(gs1[n])
    plt.hist(del_TS[n][0], bins=10, alpha=0.5)
    plt.xlabel('Del_TS\nl=0')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs2 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[1])
for n in range(5):
    ax = fig.add_subplot(gs2[n])
    plt.hist(del_TS[n][1], bins=10, alpha=0.5)
    plt.xlabel('Del_TS\nl=0.02')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs3 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[2])
for n in range(5):
    ax = fig.add_subplot(gs3[n])
    plt.hist(del_TS[n][2], bins=10, alpha=0.5)
    plt.xlabel('Del_TS\nl=0.05')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs4 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[3])
for n in range(5):
    ax = fig.add_subplot(gs4[n])
    plt.hist(del_TS[n][3], bins=10, alpha=0.5)
    plt.xlabel('Del_TS\nl=0.1')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs5 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[4])
for n in range(5):
    ax = fig.add_subplot(gs5[n])
    plt.hist(del_TS[n][4], bins=10, alpha=0.5)
    plt.xlabel('Del_TS\nl=0.5')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
plt.show()

In [ ]:
fig = plt.figure(figsize=[15,20],constrained_layout=True)

import matplotlib.gridspec as gridspec

gs0 = gridspec.GridSpec(1, 5, figure=fig)

gs1 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[0])
for n in range(5):
    ax = fig.add_subplot(gs1[n])
    plt.hist(amplitude1[n][0], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["amplitude"].value, color="green")
    plt.xlabel('Amplitude\nl=0')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs2 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[1])
for n in range(5):
    ax = fig.add_subplot(gs2[n])
    plt.hist(amplitude1[n][1], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["amplitude"].value, color="green")
    plt.xlabel('Amplitude\nl=0.02')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs3 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[2])
for n in range(5):
    ax = fig.add_subplot(gs3[n])
    plt.hist(amplitude1[n][2], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["amplitude"].value, color="green")
    plt.xlabel('Amplitude\nl=0.05')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs4 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[3])
for n in range(5):
    ax = fig.add_subplot(gs4[n])
    plt.hist(amplitude1[n][3], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["amplitude"].value, color="green")
    plt.xlabel('Amplitude\nl=0.1')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs5 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[4])
for n in range(5):
    ax = fig.add_subplot(gs5[n])
    plt.hist(amplitude1[n][4], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["amplitude"].value, color="green")
    plt.xlabel('Amplitude\nl=0.5')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
plt.show()

In [ ]:
fig = plt.figure(figsize=[15,20],constrained_layout=True)

import matplotlib.gridspec as gridspec

gs0 = gridspec.GridSpec(1, 5, figure=fig)

gs1 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[0])
for n in range(5):
    ax = fig.add_subplot(gs1[n])
    plt.hist(amplitude2[n][0], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["amplitude"].value, color="green")
    plt.xlabel('Amplitude\nl=0')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs2 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[1])
for n in range(5):
    ax = fig.add_subplot(gs2[n])
    plt.hist(amplitude2[n][1], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["amplitude"].value, color="green")
    plt.xlabel('Amplitude\nl=0.02')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs3 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[2])
for n in range(5):
    ax = fig.add_subplot(gs3[n])
    plt.hist(amplitude2[n][2], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["amplitude"].value, color="green")
    plt.xlabel('Amplitude\nl=0.05')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs4 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[3])
for n in range(5):
    ax = fig.add_subplot(gs4[n])
    plt.hist(amplitude2[n][3], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["amplitude"].value, color="green")
    plt.xlabel('Amplitude\nl=0.1')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs5 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[4])
for n in range(5):
    ax = fig.add_subplot(gs5[n])
    plt.hist(amplitude2[n][4], bins=10, alpha=0.5)
    plt.axvline(x=sim.parameters["amplitude"].value, color="green")
    plt.xlabel('Amplitude\nl=0.5')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
plt.show()

In [ ]:
fig = plt.figure(figsize=[15,20],constrained_layout=True)

import matplotlib.gridspec as gridspec

gs0 = gridspec.GridSpec(1, 5, figure=fig)

gs1 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[0])
for n in range(5):
    ax = fig.add_subplot(gs1[n])
    plt.hist(lambda_2[n][0], bins=10, alpha=0.5)
    plt.axvline(x=model[n][0].parameters["lambda_"].value, color="green")
    plt.xlabel('Lambda\nl=0')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs2 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[1])
for n in range(5):
    ax = fig.add_subplot(gs2[n])
    plt.hist(lambda_2[n][1], bins=10, alpha=0.5)
    plt.axvline(x=model[n][1].parameters["lambda_"].value, color="green")
    plt.xlabel('Lambda\nl=0.02')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs3 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[2])
for n in range(5):
    ax = fig.add_subplot(gs3[n])
    plt.hist(lambda_2[n][2], bins=10, alpha=0.5)
    plt.axvline(x=model[n][2].parameters["lambda_"].value, color="green")
    plt.xlabel('Lambda\nl=0.05')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
gs4 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[3])
for n in range(5):
    ax = fig.add_subplot(gs4[n])
    plt.hist(lambda_2[n][3], bins=10, alpha=0.5)
    plt.axvline(x=model[n][3].parameters["lambda_"].value, color="green")
    plt.xlabel('Lambda\nl=0.1')
    plt.ylabel(f'Livetime:{livetime[n]}')

gs5 = gridspec.GridSpecFromSubplotSpec(5, 1, subplot_spec=gs0[4])
for n in range(5):
    ax = fig.add_subplot(gs5[n])
    plt.hist(lambda_2[n][4], bins=10, alpha=0.5)
    plt.axvline(x=model[n][4].parameters["lambda_"].value, color="green")
    plt.xlabel('Lambda\nl=0.5')
    plt.ylabel(f'Livetime:{livetime[n]}')
    
plt.show()